In [22]:
from pathlib import Path
import cv2
import numpy as np

# โหลดภาพและตรวจสอบ
img = cv2.imread("C:/Users/Impan/Documents/ocr-engine-python/data/test_images/text_2.png")
if img is None:
    raise Exception("ไม่พบภาพที่ระบุ")

output_folder = Path("C:/Users/Impan/Documents/ocr-engine-python/data/test_images/output")
output_folder.mkdir(exist_ok=True)

# แปลงเป็น grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

binary_gaussian = cv2.adaptiveThreshold(
    gray, 
    maxValue=255, 
    adaptiveMethod=cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
    thresholdType=cv2.THRESH_BINARY_INV, 
    blockSize=51, 
    C=21 #21
)

cv2.imwrite(f"{output_folder}/binary_gaussian.png", binary_gaussian)

skeleton = cv2.ximgproc.thinning(binary_gaussian, thinningType=cv2.ximgproc.THINNING_ZHANGSUEN)

kernel = np.ones((3, 3), np.uint8)
kernel_closing = np.ones((15, 1), np.uint8)
dummy_image = cv2.dilate(skeleton, kernel, iterations=1)
closing = cv2.morphologyEx(dummy_image, cv2.MORPH_CLOSE, kernel=kernel_closing, iterations=1)

cv2.imwrite(f"{output_folder}/closing.png", closing)

# หา contours จากภาพ binary
#contours, hierarchy = cv2.findContours(binary_gaussian, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours, hierarchy = cv2.findContours(closing, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnt = max(contours, key=cv2.contourArea)

# สร้าง mask ที่มีขนาดเดียวกับภาพ (ค่าเริ่มต้นเป็น 0, สีดำ)
mask = np.zeros(img.shape[:2], dtype=np.uint8)

# วาด contour ลงใน mask โดยเติมเต็ม (thickness = -1) ให้ภายใน contour เป็นสีขาว (255)
cv2.drawContours(mask, [cnt], -1, 255, -1)

# ใช้ mask กับภาพต้นฉบับ เพื่อดึงเฉพาะส่วนภายใน contour
result = cv2.bitwise_and(binary_gaussian, binary_gaussian, mask=mask)

inverted_img = cv2.bitwise_not(result)

# แสดงผลลัพธ์
cv2.imshow("Original", img)
cv2.imshow("Mask", mask)
cv2.imshow("Cropped Object with Black Background", result)
cv2.imshow("inverted_img", inverted_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

'''
# วนลูปผ่านแต่ละ contour
for idx, cnt in enumerate(contours):
    # กรอง contour ที่มีพื้นที่เล็กเกินไป (อาจเป็น noise)
    if cv2.contourArea(cnt) < 1:
        continue

    # สร้าง mask ที่มีขนาดเท่ากับภาพ (ค่าเริ่มต้นเป็น 0)
    mask = np.zeros_like(gray)

    # วาด contour ลงบน mask โดยเติมเต็มด้วยสีขาว (255)
    char_mask = cv2.drawContours(mask, [cnt], -1, 255, -1)
    cv2.imwrite(f"{output_folder}/char_mask_{idx}.png", char_mask)


    
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(char_mask, connectivity=8)
    areas = [stat[4] for stat in stats]  # ดึงค่า area
    sorted_areas = sorted(areas, reverse=True)  # เรียงลำดับจากมากไปน้อย
    second_max_area = sorted_areas[1]  # ค่าอันดับ 2
    second_max_area_index = areas.index(second_max_area)  # หาตำแหน่งในลิสต์เดิม
    char_position = stats[second_max_area_index]
    x, y, w, h, area = char_position
    

    # ใช้ mask ดึงตัวอักษรออกมาจากภาพ (เก็บเฉพาะส่วนที่เป็น white ใน mask)
    #letter = cv2.bitwise_and(img, img, mask=mask)

    # หากต้องการ crop ให้แคบลงตามพื้นที่จริง
    x, y, w, h = cv2.boundingRect(cnt)
    #letter_cropped = letter[y:y+h, x:x+w]

    #cv2.imshow("Letter", letter_cropped)
    #cv2.waitKey(0)
    #cv2.imwrite(f"{output_folder}/Letter_{idx}.png", letter_cropped)
    
'''
#cv2.destroyAllWindows()

'\n# วนลูปผ่านแต่ละ contour\nfor idx, cnt in enumerate(contours):\n    # กรอง contour ที่มีพื้นที่เล็กเกินไป (อาจเป็น noise)\n    if cv2.contourArea(cnt) < 1:\n        continue\n\n    # สร้าง mask ที่มีขนาดเท่ากับภาพ (ค่าเริ่มต้นเป็น 0)\n    mask = np.zeros_like(gray)\n\n    # วาด contour ลงบน mask โดยเติมเต็มด้วยสีขาว (255)\n    char_mask = cv2.drawContours(mask, [cnt], -1, 255, -1)\n    cv2.imwrite(f"{output_folder}/char_mask_{idx}.png", char_mask)\n\n\n    \n    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(char_mask, connectivity=8)\n    areas = [stat[4] for stat in stats]  # ดึงค่า area\n    sorted_areas = sorted(areas, reverse=True)  # เรียงลำดับจากมากไปน้อย\n    second_max_area = sorted_areas[1]  # ค่าอันดับ 2\n    second_max_area_index = areas.index(second_max_area)  # หาตำแหน่งในลิสต์เดิม\n    char_position = stats[second_max_area_index]\n    x, y, w, h, area = char_position\n    \n\n    # ใช้ mask ดึงตัวอักษรออกมาจากภาพ (เก็บเฉพาะส่วนที่เป็น white ใน